# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [86]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talkurl, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [87]:
!cat talks.tsv

title	type	url_slug	venue	date	location	talk_url	description	full_abstract
Voluntary and involuntary autobiographical memories: More similar than different?	Talk	2022-CSBBCS	CSBBCS	x2022-6-19	"Halifax, NS"		"In everyday life, memories of one's personal past (autobiographical memories; AMs) are typically retrieved either voluntarily or involuntarily. Here, we examined similarities and differences between voluntary and involuntary AMs (N = 4,801). Results indicated that recurrent IAMs were largely distinguished by higher emotionality/importance and lower vividness than voluntary AMs, though both types of memory overlapped considerably with regard to memory properties. Importantly, the emotionality of recurrent IAMs predicted symptoms of mental health disorders significantly better than voluntary AMs. Our work supports modeling recurrent IAMs as general cognitive phenomena with unique links to mental health."	"In everyday life, memories of one's personal past (autobiographical memories; A

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [88]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description,full_abstract
0,Voluntary and involuntary autobiographical mem...,Talk,2022-CSBBCS,CSBBCS,x2022-6-19,"Halifax, NS",NaN,"In everyday life, memories of one's personal p...","In everyday life, memories of one's personal p..."
1,Assessing impacts of Waterloo's undergraduate ...,Talk (co-author),2021-UWTL,University of Waterloo,x2021-4-27,"Waterloo, ON",NaN,"In 2020, the Faculty of Health launched the Un...",Concerns about post-secondary students' mental...
2,Key takeaways from Waterloo's undergraduate me...,Talk (co-author),2021-UWSMHR,University of Waterloo,x2021-11-10,"Waterloo, ON",NaN,"Since Winter 2020, the Faculty of Health has o...","Since Winter 2020, the Faculty of Health has o..."
3,Recurrent involuntary memories are linked to m...,Talk (co-author),2021-MDRS,MDRS,x2021-10-5,Virtual,NaN,Recurrent memories are linked to individual di...,NaN
4,What are recurrent memories about? Understandi...,Talk,2021-CSBBCS,CSBBCS,x2021-6-17,"Montreal, QC",https://youtu.be/ljiLS8pEfgU,Researchers debate whether recurrent involunta...,Researchers debate whether recurrent involunta...
5,Assessing Waterloo's first undergraduate cours...,Talk,2020-UWSMHR,University of Waterloo,x2020-11-5,"Waterloo, ON",https://web.archive.org/web/20210411054156/htt...,Does completion of a course on MHL have an eff...,"Purpose: In Winter 2020, the University of Wat..."
6,University of Waterloo's undergraduate course ...,Talk,2020-CICMH,Centre for Innovation in Campus Mental Health,x2020-11-3,"Toronto, ON",NaN,We describe the design and evaluation of the U...,"In this evidence-informed session, we describe..."
7,Anxiety-provoking context scenes can reduce th...,Talk (co-author),2019b-CSBBCS,CSBBCS,x2019-6-9,"Waterloo, ON",NaN,Reinstating the same context from encoding dur...,"Reinstating the same context during retrieval,..."
8,From tea cakes to trauma: Bridging gaps betwee...,Talk,2019a-CSBBCS,CSBBCS,x2019-6-8,"Waterloo, ON",NaN,We investigated the properties of recurrent IA...,Memories of one's personal past that come to m...
9,From tea cakes to trauma: Bridging gaps betwee...,Talk,2019-TAMeG,TAMeG,x2019-5-14,"Toronto, ON",NaN,Are recurrent memories common in the general p...,NaN


RY: Added a random character ("x") to the start of each date in talks.tsv to prevent Excel from converting the column to date format and breaking the markdown files. Following chunk deletes the first character and restores to appropriate YYYY-MM-DD format.

In [89]:
talks['date'] = talks['date'].str[1:]
talks

,title,type,url_slug,venue,date,location,talk_url,description,full_abstract
0,Voluntary and involuntary autobiographical mem...,Talk,2022-CSBBCS,CSBBCS,2022-6-19,"Halifax, NS",NaN,"In everyday life, memories of one's personal p...","In everyday life, memories of one's personal p..."
1,Assessing impacts of Waterloo's undergraduate ...,Talk (co-author),2021-UWTL,University of Waterloo,2021-4-27,"Waterloo, ON",NaN,"In 2020, the Faculty of Health launched the Un...",Concerns about post-secondary students' mental...
2,Key takeaways from Waterloo's undergraduate me...,Talk (co-author),2021-UWSMHR,University of Waterloo,2021-11-10,"Waterloo, ON",NaN,"Since Winter 2020, the Faculty of Health has o...","Since Winter 2020, the Faculty of Health has o..."
3,Recurrent involuntary memories are linked to m...,Talk (co-author),2021-MDRS,MDRS,2021-10-5,Virtual,NaN,Recurrent memories are linked to individual di...,NaN
4,What are recurrent memories about? Understandi...,Talk,2021-CSBBCS,CSBBCS,2021-6-17,"Montreal, QC",https://youtu.be/ljiLS8pEfgU,Researchers debate whether recurrent involunta...,Researchers debate whether recurrent involunta...
5,Assessing Waterloo's first undergraduate cours...,Talk,2020-UWSMHR,University of Waterloo,2020-11-5,"Waterloo, ON",https://web.archive.org/web/20210411054156/htt...,Does completion of a course on MHL have an eff...,"Purpose: In Winter 2020, the University of Wat..."
6,University of Waterloo's undergraduate course ...,Talk,2020-CICMH,Centre for Innovation in Campus Mental Health,2020-11-3,"Toronto, ON",NaN,We describe the design and evaluation of the U...,"In this evidence-informed session, we describe..."
7,Anxiety-provoking context scenes can reduce th...,Talk (co-author),2019b-CSBBCS,CSBBCS,2019-6-9,"Waterloo, ON",NaN,Reinstating the same context from encoding dur...,"Reinstating the same context during retrieval,..."
8,From tea cakes to trauma: Bridging gaps betwee...,Talk,2019a-CSBBCS,CSBBCS,2019-6-8,"Waterloo, ON",NaN,We investigated the properties of recurrent IA...,Memories of one's personal past that come to m...
9,From tea cakes to trauma: Bridging gaps betwee...,Talk,2019-TAMeG,TAMeG,2019-5-14,"Toronto, ON",NaN,Are recurrent memories common in the general p...,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [90]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [91]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 2:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.date)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    if len(str(item.talk_url)) > 3:
        md += "talkurl: " + str(item.talk_url) + "\n"
        #md += "\n[More information here](" + item.talkurl + ")\n" 

    md += "---\n"
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [92]:
!ls ../_talks

2018-10-6-2018-MDRS.md    2019-6-9-2019b-CSBBCS.md  2021-4-27-2021-UWTL.md
2018-5-15-2018-TAMeG.md   2020-11-3-2020-CICMH.md   2021-6-17-2021-CSBBCS.md
2019-2-23-2019-INS.md     2020-11-5-2020-UWSMHR.md  2022-6-19-2022-CSBBCS.md
2019-5-14-2019-TAMeG.md   2021-10-5-2021-MDRS.md
2019-6-8-2019a-CSBBCS.md  2021-11-10-2021-UWSMHR.md


In [93]:
!cat ../_talks/2013-03-01-tutorial-1.md

cat: ../_talks/2013-03-01-tutorial-1.md: No such file or directory
